In [6]:
'''
    A técnica de augmentation busca aumentar a quantidade de imagens na base de dados, criando 
    variações da imagens existentes no bd, com varições de zoom, tamanho, rotação, etc.
    O Augumentation auxilia situações em que a base de dados para o treinamento não é extensa.
'''
from keras.datasets import mnist  # Importação da base de dados
from keras.models import Sequential  # Modelo sequencial
from keras.layers import Dense, Flatten  # 3ª e 4ª etapa da CNN
from keras.layers import Conv2D, MaxPooling2D  # 1ª e 2ª etapa da CNN
from keras.layers import BatchNormalization
from keras import utils
from keras.preprocessing.image import ImageDataGenerator

In [4]:
'''
    * Importação dos dados
    * Redimensionamento das imagens (obter apenas 1 camada)
    * Transformação do tipo de dado para a normalização
    * Normalização
    * Criação das dummy variables para as classes de treinamento e teste
'''
(xTrain, yTrain), (xTest, yTest) = mnist.load_data()

trainForecasters = xTrain.reshape(xTrain.shape[0], 28, 28, 1)
testForecasters = xTest.reshape(xTest.shape[0], 28, 28, 1)
trainForecasters = trainForecasters.astype('float32')
testForecasters = testForecasters.astype('float32')
trainForecasters /= 255
testForecasters /= 255

trainClass = utils.to_categorical(yTrain, 10)
testClass = utils.to_categorical(yTest, 10)

In [5]:
'''
    * Criação da cnn no modelo sequencial (sequencia de layers)
    * Camada de convolução com função de ativação Relu
    * Normalização dos mapas de características
    * Processo de max Pooling
    * Processo de flattening 
    * Adição de uma hidden layer
    * Adição da camada de saída com função de ativação 'softmax'
    * Compilação da rede neural
'''

classifier = Sequential()
classifier.add(Conv2D(32, (3,3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Flatten())

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 10, activation = 'softmax'))
classifier.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [7]:
'''
    Geração das novas imagens, pelo processo de augmentation
    Inicialmente é necessário gerar variáveis dotadas das definições 
    de como as imagens serão modificadas pelo Image data Generator 
    (Biblioteca do keras responsável por aplicar
    alterações nas imagens do dataset a fim de ampliá-lo)

    * generatedTrain -> contem as definições dos dados para treinamento
    * generatedTest -> contém as definições dos dados para teste

    Explicação dos parâmetros utilizados:

    * rotation_range -> Valor em graus (0-180), um intervalo dentro do qual as imagens podem ser rotacionadas aleatoriamente
    * horizontal_flip -> Inverter aleatoriamente metade das imagens horizontalmente
    * shear_range -> Aplicar transformações de cisalhamento aleatoriamente
    * zoom_range -> Valor ou intervalo para definir um zoom aleatório aplicado
    * height_shift_range -> Intervalos (como uma fração da altura total) dentro dos quais traduzem 
      imagens aleatoriamente na vertical 

    Observação: 'generatedTest' não recebe nenhum parâmetro uma vez que 
    não serão feitas transformações da imagem. A intenção do aumento de imagens
    é exclusiva para o treinamento, as imagens de teste se adaptam às imagens novas
'''

generatedTrain = ImageDataGenerator(rotation_range = 7,
                                    horizontal_flip = True,
                                    shear_range = 0.2,
                                    height_shift_range = 0.07,
                                    zoom_range = 0.2)

generatedTest = ImageDataGenerator()

'''
    Uma nova base de dados é gerada a partir dos dados pré-existentes, aplicando
    as definições descritas previamentes pela classe ImageDataGenerator.
    Observação, utilizando a técnica de augmentation, o batch size passa a ser definido aqui.
'''

trainDatabase = generatedTrain.flow(trainForecasters, trainClass, batch_size = 128)
testDatabase = generatedTest.flow(testForecasters, testClass, batch_size =128)

'''
    Etapa de treinamento da rede neural. É importante que a função 'fit_generator' está sendo
    utilizada ao invés da função 'fit' uma vez que ela suporta o processo de augmentation, contudo
    tal função está em processo de depreciação, uma vez que em versões mais atuais, a função 'fit' 
    também suporta.

    Explicação dos parâmetros:

    * trainDatabase -> Dados para treino (após a augmentation)
    * steps_per_epoch -> Número total de etapas (lotes de amostras) a serem produzidas 
                         pelo gerador antes de declarar uma época concluída e iniciar a próxima época.
                         É importante citar que o valor ideal para este parâmetro se dá pelo total de 
                         amostras dividido pel valor do batch_size 
    * epochs -> Epocas de treinamento da rede
    * validation_data -> Dados para a validação (após a augmentation)
    * validation_steps -> Possui o mesmo princípio do 'steps_per_epoch', porém levando em 
                          consideração a etapa de validação. O valor ideal para este parâmetro 
                          se dá pelo total de amostras dividido pel valor do batch_siz e
'''

classifier.fit_generator(trainDatabase, 
                         steps_per_epoch = 60000/128, 
                         epochs = 5, 
                         validation_data = testDatabase,
                         validation_steps = 10000 / 128)



C:\Users\marci\AppData\Local\Temp\ipykernel_20456\164371016.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(trainDatabase,


Epoch 1/5


468/468 [==============================] - 20s 41ms/step - loss: 0.3138 - accuracy: 0.9001 - val_loss: 0.2246 - val_accuracy: 0.9605
Epoch 2/5
468/468 [==============================] - 18s 38ms/step - loss: 0.1413 - accuracy: 0.9563 - val_loss: 0.0899 - val_accuracy: 0.9729
Epoch 3/5
468/468 [==============================] - 18s 39ms/step - loss: 0.1115 - accuracy: 0.9656 - val_loss: 0.1059 - val_accuracy: 0.9678
Epoch 4/5
468/468 [==============================] - 18s 39ms/step - loss: 0.0999 - accuracy: 0.9687 - val_loss: 0.0738 - val_accuracy: 0.9791
Epoch 5/5
468/468 [==============================] - 20s 42ms/step - loss: 0.0901 - accuracy: 0.9716 - val_loss: 0.0853 - val_accuracy: 0.9737
